# SpaceX Falcon 9 First Stage Landing Prediction
## Part 7: Dashboard Application with Plotly Dash
***
### Objectives
*   Create dashboard using Plotly Dash
*   Populate dashboard with dropdown and slider elements
***
### Import Libraries

In [1]:
!pip install dash
!pip install plotly
!pip install jupyter-dash

In [2]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px

### Build Dashboard

In [3]:
# Get data from part 5
spacex_df = pd.read_csv('03_dataset.csv')
spacex_df = spacex_df[['FlightNumber', 'LaunchSite', 'Class', 'PayloadMass', 'Serial']]
max_payload = spacex_df['PayloadMass'].max()
min_payload = spacex_df['PayloadMass'].min()
spacex_df.head()

,FlightNumber,LaunchSite,Class,PayloadMass,Serial
0,1,CCSFS SLC 40,0,7197.514904,B0003
1,2,CCSFS SLC 40,0,525.000000,B0005
2,3,CCSFS SLC 40,0,677.000000,B0007
3,4,VAFB SLC 4E,0,500.000000,B1003
4,5,CCSFS SLC 40,0,3170.000000,B1004


In [4]:
app = JupyterDash(__name__)

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                dcc.Dropdown(id='site-dropdown', 
                                                options=[{'label': 'All Sites', 'value': 'ALL'},
                                                        {'label':'VAFB SLC-4E', 'value': 'VAFB SLC 4E'},
                                                        {'label':'KSC LC-39A', 'value': 'KSC LC 39A'},
                                                        {'label':'CCSFS SLC-40', 'value': 'CCSFS SLC 40'}], 
                                                value='ALL',
                                                placeholder='Select Launch Site', 
                                                searchable=True),
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (kg):"),
                                dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000, value=[0, 10000], 
                                                marks={0:'0', 1000:'1000', 2000:'2000', 3000:'3000', 4000:'4000', 5000:'5000', 
                                                        6000:'6000', 7000:'7000', 8000:'8000', 9000:'9000', 10000:'10000'}),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

@app.callback(Output(component_id='success-pie-chart', component_property='figure'), 
                Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['LaunchSite'] == entered_site].groupby(['Class']).count().reset_index()
    filtered_df.rename(columns={'FlightNumber': 'count'}, inplace=True)
    if entered_site =='ALL':
        fig = px.pie(spacex_df, values='Class', names='LaunchSite', title='Total Successful Launches by Site')
        return fig
    else:
        fig = px.pie(filtered_df, values ='count', names='Class', color='Class', color_discrete_sequence=['red', 'green'], title='Total Successful Launches for Site '+ entered_site)
        return fig

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                [Input(component_id='site-dropdown', component_property='value'), 
                Input(component_id='payload-slider', component_property='value')])

def get_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    filtered2_df = spacex_df[spacex_df['LaunchSite'] == entered_site].reset_index()
    if entered_site =='ALL':
        mask = (spacex_df['PayloadMass'] >= low) & (spacex_df['PayloadMass'] <= high)
        fig = px.scatter(spacex_df[mask], x='PayloadMass', y='Class', 
                            color='LaunchSite', range_x=[low, high],
                            title='Relationship Between Payload Mass and Landing Success for All Sites')
        return fig
    else:
        mask = (filtered2_df['PayloadMass'] >= low) & (filtered2_df['PayloadMass'] <= high)
        fig = px.scatter(filtered2_df[mask], x='PayloadMass', y='Class',
                            color='LaunchSite', range_x=[low, high],
                            title='Relationship Between Payload Mass and Landing Success for '+ entered_site)
        return fig

app.run_server(mode='inline')